<a href="https://colab.research.google.com/github/ritesh-chafer/Coronavirus_Enrichment/blob/master/Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls '/content/drive/My Drive/Colab_Notebooks/twitter.properties'

'/content/drive/My Drive/Colab_Notebooks/twitter.properties'


In [3]:
!pip3 install ConfigParser

In [4]:
import configparser
config = configparser.RawConfigParser()
config.read('//content//drive//My Drive//Colab_Notebooks//twitter.properties')
print(config.sections())

['twitter']


In [5]:
import pandas as pd
import numpy as np
import os
import tweepy as tw

In [6]:
access_token=config.get('twitter', 'accesstoken')
access_tokensecret=config.get('twitter', 'accesstokensecret')
api_key=config.get('twitter', 'apikey')
api_secretkey=config.get('twitter', 'apisecretkey')
map_api_key = config.get('twitter', 'googleapikey')

In [8]:
# Set up the auth tokens
auth = tw.OAuthHandler(api_key, api_secretkey)
auth.set_access_token(access_token, access_tokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [9]:
search_words = '#coronavirus'
date_since = '2020-3-10'

In [14]:
tweets = tw.Cursor(api.search,q=search_words,since=date_since,lang="en").items(1000)

In [15]:
tweets

In [16]:
from __future__ import unicode_literals

In [17]:
#tweet_details
tweet_details = [[tweet.geo,tweet.text,tweet.user.screen_name,tweet.user.location] for tweet in tweets]

In [18]:
tweet_df = pd.DataFrame(data=tweet_details,columns=['geo','text','user',"location"])

In [19]:
pd.set_option('max_colwidth', 800)
tweet_df.head(20)

,geo,text,user,location
0,None,#BIGNEWS - #Mumbai today recorded less than 100 #COVID19 cases today. It reported 96 new cases &amp; 01 death in the la… https://t.co/pJWR873PEw,disha2791,Mumbai
1,None,"Art has always been a mirror of #society &amp; societal change. In a world wading through #Coronavirus updates, contemp… https://t.co/EzKSxsY9bF",30stades,"Mumbai, India"
2,None,RT @bianco222: We might all die. But we'll do it with dignity! #NoPinappleOnPizza\n#coronavirus #covid_19italia #Italia #italylockdown https…,Fe_geso256,Thousand Leaves Prefecture
3,None,Click on the link below to read the full news\n👇👇\nhttps://t.co/RkKlKoe5HV...\n..........\n@arthparkash1… https://t.co/nwszYoOLqB,arthparkash1,India
4,None,"RT @ProtestNews_EN: The voice of freedom, people of the Styria, Austria, took to the streets last night to be the first step of their free…",wavetossed,"Vancouver, BC"
5,None,"RT @Zeinobia: A New-old thread: #ExperienceEgyptfromHome \n#Stuckathome coz of #Coronavirus, then take a tour in #Egypt's Kodak Agfa's fines…",NPSusa,"Washington, D.C."
6,None,@JMPSimor We should also ask about how many #laws are in place and laid in parliament to do with #coronavirus ! Luc… https://t.co/AZvUoBo5F1,samdownie,"Films, TV, Radio & Music"
7,None,#Jesus Saves\n\n#MAGA #LSU #Louisiana #batonrouge LSU #covid19 #covid #coronavirus Covid-19 #nola #maga Baton Rouge… https://t.co/TyaOgQKlR9,dreamchqser,"Zachary, LA"
8,None,This is a summary of the Novel #Coronavirus outbreak in #Tochigi until today. (22/2/21)\n\n■Current utilization of be… https://t.co/niEE1HOahs,Tobucchi_Inter,Tochigi / とちぎけん
9,None,RT @dimension815: ⚡️ #Sunday in #Canada is still in full swing and cities are filled with #protesters against #coronavirus #restrictions\n\nP…,CarolAtherton9,


In [20]:
tweet_df.user.value_counts()

bitsnewsindia     25
__headspaceAI     12
corona_tweet      12
_covid_19_bot_    12
Z3itG3ist         12
                  ..
myGoldharts        1
Chana_Chemist      1
DianaCheung1       1
ocean20182018      1
LivingTheWord7     1
Name: user, Length: 750, dtype: int64

In [21]:
tweet_df.location.value_counts()

                                  317
India                              46
United States                      17
Sainte-Barbe-de-la-Providence´     12
In your head.                      12
                                 ... 
Gainesville, FL                     1
Vector of Disease                   1
Kaduna, Nigeria                     1
Yorkshire                           1
Southern California                 1
Name: location, Length: 410, dtype: int64

In [22]:
import re
def clean_tweets(text):
  text = re.sub("RT @[\w]*:", "", text)
  text = re.sub("@[\w]*", "", text)
  text = re.sub("https?://[A-Za-z0-9./]*", "", text)
  text = re.sub("\n", "", text)
  return text

In [23]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_tweets(x))

In [24]:
tweet_df.head(20)

,geo,text,user,location
0,None,#BIGNEWS - #Mumbai today recorded less than 100 #COVID19 cases today. It reported 96 new cases &amp; 01 death in the la…,disha2791,Mumbai
1,None,"Art has always been a mirror of #society &amp; societal change. In a world wading through #Coronavirus updates, contemp…",30stades,"Mumbai, India"
2,None,We might all die. But we'll do it with dignity! #NoPinappleOnPizza#coronavirus #covid_19italia #Italia #italylockdown https…,Fe_geso256,Thousand Leaves Prefecture
3,None,Click on the link below to read the full news👇👇..........…,arthparkash1,India
4,None,"The voice of freedom, people of the Styria, Austria, took to the streets last night to be the first step of their free…",wavetossed,"Vancouver, BC"
5,None,"A New-old thread: #ExperienceEgyptfromHome #Stuckathome coz of #Coronavirus, then take a tour in #Egypt's Kodak Agfa's fines…",NPSusa,"Washington, D.C."
6,None,We should also ask about how many #laws are in place and laid in parliament to do with #coronavirus ! Luc…,samdownie,"Films, TV, Radio & Music"
7,None,#Jesus Saves#MAGA #LSU #Louisiana #batonrouge LSU #covid19 #covid #coronavirus Covid-19 #nola #maga Baton Rouge…,dreamchqser,"Zachary, LA"
8,None,This is a summary of the Novel #Coronavirus outbreak in #Tochigi until today. (22/2/21)■Current utilization of be…,Tobucchi_Inter,Tochigi / とちぎけん
9,None,⚡️ #Sunday in #Canada is still in full swing and cities are filled with #protesters against #coronavirus #restrictionsP…,CarolAtherton9,


In [25]:
tweet_df.to_csv('tweets.csv')

In [26]:
!ls

drive  sample_data  tweets.csv


In [27]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [28]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text,ent.label_)) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

	Text : BIGNEWS, Entity : PERSON
	Text : Mumbai, Entity : GPE
	Text : today, Entity : DATE
	Text : less than 100 #COVID19, Entity : MONEY
	Text : today, Entity : DATE
	Text : 96, Entity : CARDINAL
	Text : 01, Entity : CARDINAL
	Text : Coronavirus, Entity : PERSON
	Text : The voice of freedom, Entity : ORG
	Text : Styria, Entity : GPE
	Text : Austria, Entity : GPE
	Text : last night, Entity : TIME
	Text : first, Entity : ORDINAL
	Text : Stuckathome, Entity : PERSON
	Text : Coronavirus, Entity : PERSON
	Text : Egypt, Entity : GPE
	Text : Kodak Agfa's, Entity : ORG
	Text : coronavirus, Entity : PERSON
	Text : Jesus, Entity : PERSON
	Text : Baton Rouge, Entity : GPE
	Text : Novel, Entity : ORG
	Text : Coronavirus, Entity : PERSON
	Text : Tochigi, Entity : PERSON
	Text : today, Entity : DATE
	Text : Sunday, Entity : DATE
	Text : Canada, Entity : GPE
	Text : MentalHealthNursesDay, Entity : PERSON
	Text : Mental Health Nurses, Entity : ORG
	Text : Andhra Pradesh Registers, Entity : PERSON
	Te

0      [None, None, None, None, None, None, None]
1                                        [, None]
2                                              []
3                                              []
4                  [None, None, None, None, None]
                          ...                    
995                                            []
996                      [None, None, None, None]
997                                  [None, None]
998    [None, None, None, None, None, None, None]
999                                      [None, ]
Name: text, Length: 1000, dtype: object

In [29]:
tweet_df['entities'] = tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [ ]:
tweet_df.head(20)

In [30]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

In [31]:
sid = SentimentIntensityAnalyzer()

In [32]:
tweet_df['sentiment'] = tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [33]:
tweet_df.head(20)

,geo,text,user,location,entities,sentiment
0,None,#BIGNEWS - #Mumbai today recorded less than 100 #COVID19 cases today. It reported 96 new cases &amp; 01 death in the la…,disha2791,Mumbai,"[(BIGNEWS, PERSON), (Mumbai, GPE), (today, DATE), (less than 100 #COVID19, MONEY), (today, DATE), (96, CARDINAL), (01, CARDINAL)]","{'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'compound': -0.5994}"
1,None,"Art has always been a mirror of #society &amp; societal change. In a world wading through #Coronavirus updates, contemp…",30stades,"Mumbai, India","[, (Coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
2,None,We might all die. But we'll do it with dignity! #NoPinappleOnPizza#coronavirus #covid_19italia #Italia #italylockdown https…,Fe_geso256,Thousand Leaves Prefecture,[],"{'neg': 0.211, 'neu': 0.654, 'pos': 0.136, 'compound': -0.3595}"
3,None,Click on the link below to read the full news👇👇..........…,arthparkash1,India,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
4,None,"The voice of freedom, people of the Styria, Austria, took to the streets last night to be the first step of their free…",wavetossed,"Vancouver, BC","[(The voice of freedom, ORG), (Styria, GPE), (Austria, GPE), (last night, TIME), (first, ORDINAL)]","{'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compound': 0.6369}"
5,None,"A New-old thread: #ExperienceEgyptfromHome #Stuckathome coz of #Coronavirus, then take a tour in #Egypt's Kodak Agfa's fines…",NPSusa,"Washington, D.C.","[(Stuckathome, PERSON), (Coronavirus, PERSON), (Egypt, GPE), (Kodak Agfa's, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
6,None,We should also ask about how many #laws are in place and laid in parliament to do with #coronavirus ! Luc…,samdownie,"Films, TV, Radio & Music","[(coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
7,None,#Jesus Saves#MAGA #LSU #Louisiana #batonrouge LSU #covid19 #covid #coronavirus Covid-19 #nola #maga Baton Rouge…,dreamchqser,"Zachary, LA","[(Jesus, PERSON), , , (Baton Rouge, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
8,None,This is a summary of the Novel #Coronavirus outbreak in #Tochigi until today. (22/2/21)■Current utilization of be…,Tobucchi_Inter,Tochigi / とちぎけん,"[(Novel, ORG), (Coronavirus, PERSON), (Tochigi, PERSON), (today, DATE)]","{'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.3182}"
9,None,⚡️ #Sunday in #Canada is still in full swing and cities are filled with #protesters against #coronavirus #restrictionsP…,CarolAtherton9,,"[(Sunday, DATE), (Canada, GPE), , ]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [37]:
!pip3 install opencage

     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 3.6 MB 46.5 MB/s 


In [38]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [39]:
geocoder = OpenCageGeocode(map_api_key)

In [43]:
results = geocoder.reverse_geocode(28.033900,	1.659600)
print(results)
print(results[0]['formatted'])
print(results[0]['formatted'].split(",")[-2].strip(" "))

[{'annotations': {'DMS': {'lat': "27° 1' 17.24844'' N", 'lng': "1° 0' 50.71572'' E"}, 'MGRS': '31RCK0298190363', 'Maidenhead': 'JL07ma15qd', 'Mercator': {'x': 112887.726, 'y': 3106745.359}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=4171628#map=17/27.02146/1.01409', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/27.02146/1.01409&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=27.02146&mlon=1.01409#map=17/27.02146/1.01409'}, 'UN_M49': {'regions': {'AFRICA': '002', 'DZ': '012', 'NORTHERN_AFRICA': '015', 'WORLD': '001'}, 'statistical_groupings': ['LEDC']}, 'callingcode': 213, 'currency': {'alternate_symbols': ['DA'], 'decimal_mark': '.', 'format': '%n %u', 'html_entity': '', 'iso_code': 'DZD', 'iso_numeric': '012', 'name': 'Algerian Dinar', 'smallest_denomination': 100, 'subunit': 'Centime', 'subunit_to_unit': 100, 'symbol': 'د.ج', 'symbol_first': 0, 'thousands_separator': ','}, 'flag': '🇩🇿', 'geohash': 'shb9xp47muydgn9nqs3s', 'qibla': 90.41, 'road

In [44]:
results = geocoder.geocode(tweet_df['location'][1])
print(results)
print(results[0]['formatted'])
print(results[0]['formatted'].split(",")[-1].strip())

[{'annotations': {'DMS': {'lat': "19° 4' 33.56364'' N", 'lng': "72° 52' 38.61408'' E"}, 'MGRS': '43QBB7665610588', 'Maidenhead': 'MK69kb58gf', 'Mercator': {'x': 8112674.257, 'y': 2149926.679}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=7888990#map=16/19.07599/72.87739', 'note_url': 'https://www.openstreetmap.org/note/new#map=16/19.07599/72.87739&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=19.07599&mlon=72.87739#map=16/19.07599/72.87739'}, 'UN_M49': {'regions': {'ASIA': '142', 'IN': '356', 'SOUTHERN_ASIA': '034', 'WORLD': '001'}, 'statistical_groupings': ['LEDC']}, 'callingcode': 91, 'currency': {'alternate_symbols': ['Rs', '৳', '૱', '௹', 'रु', '₨'], 'decimal_mark': '.', 'html_entity': '&#x20b9;', 'iso_code': 'INR', 'iso_numeric': '356', 'name': 'Indian Rupee', 'smallest_denomination': 50, 'subunit': 'Paisa', 'subunit_to_unit': 100, 'symbol': '₹', 'symbol_first': 1, 'thousands_separator': ','}, 'flag': '🇮🇳', 'geohash': 'te7ud2etwq68nj63wnwm', 'qibla': 28

In [49]:
def get_country(input):
  try:
    output = geocoder.geocode(input)[0]['formatted'].split(",")[-1].strip()
    
  except:
    output="Error"
    
  return output

In [50]:
tweet_df['country']=tweet_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

In [51]:
tweet_df['country'].value_counts()

                            318
United Kingdom              150
India                        98
United States of America     84
Error                        39
                           ... 
Bangladesh                    1
Turkey                        1
Russia                        1
Denmark                       1
Georgia                       1
Name: country, Length: 62, dtype: int64

In [52]:
tweet_df.head(10)

,geo,text,user,location,entities,sentiment,country
0,None,#BIGNEWS - #Mumbai today recorded less than 100 #COVID19 cases today. It reported 96 new cases &amp; 01 death in the la…,disha2791,Mumbai,"[(BIGNEWS, PERSON), (Mumbai, GPE), (today, DATE), (less than 100 #COVID19, MONEY), (today, DATE), (96, CARDINAL), (01, CARDINAL)]","{'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'compound': -0.5994}",India
1,None,"Art has always been a mirror of #society &amp; societal change. In a world wading through #Coronavirus updates, contemp…",30stades,"Mumbai, India","[, (Coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",India
2,None,We might all die. But we'll do it with dignity! #NoPinappleOnPizza#coronavirus #covid_19italia #Italia #italylockdown https…,Fe_geso256,Thousand Leaves Prefecture,[],"{'neg': 0.211, 'neu': 0.654, 'pos': 0.136, 'compound': -0.3595}",United Kingdom
3,None,Click on the link below to read the full news👇👇..........…,arthparkash1,India,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",India
4,None,"The voice of freedom, people of the Styria, Austria, took to the streets last night to be the first step of their free…",wavetossed,"Vancouver, BC","[(The voice of freedom, ORG), (Styria, GPE), (Austria, GPE), (last night, TIME), (first, ORDINAL)]","{'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compound': 0.6369}",Canada
5,None,"A New-old thread: #ExperienceEgyptfromHome #Stuckathome coz of #Coronavirus, then take a tour in #Egypt's Kodak Agfa's fines…",NPSusa,"Washington, D.C.","[(Stuckathome, PERSON), (Coronavirus, PERSON), (Egypt, GPE), (Kodak Agfa's, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",United States of America
6,None,We should also ask about how many #laws are in place and laid in parliament to do with #coronavirus ! Luc…,samdownie,"Films, TV, Radio & Music","[(coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Error
7,None,#Jesus Saves#MAGA #LSU #Louisiana #batonrouge LSU #covid19 #covid #coronavirus Covid-19 #nola #maga Baton Rouge…,dreamchqser,"Zachary, LA","[(Jesus, PERSON), , , (Baton Rouge, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",United States of America
8,None,This is a summary of the Novel #Coronavirus outbreak in #Tochigi until today. (22/2/21)■Current utilization of be…,Tobucchi_Inter,Tochigi / とちぎけん,"[(Novel, ORG), (Coronavirus, PERSON), (Tochigi, PERSON), (today, DATE)]","{'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.3182}",Japan
9,None,⚡️ #Sunday in #Canada is still in full swing and cities are filled with #protesters against #coronavirus #restrictionsP…,CarolAtherton9,,"[(Sunday, DATE), (Canada, GPE), , ]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",


In [53]:
tweet_df.dtypes

geo          object
text         object
user         object
location     object
entities     object
sentiment    object
country      object
dtype: object

In [54]:
tweet_df.to_csv("tweets.csv")